In [ ]:
import numpy as np
import pandas as pd

df_original = pd.read_csv("original_dataset_credit_score_corrected.csv")

In [ ]:
X2 = df_original.drop('Loan Status',1)
Y2 = df_original['Loan Status']

In [ ]:
###########################################Creating the new csv with Feature Tools created features ##############
import sklearn.feature_selection 

#Such a large set of features may cause over fitting  and also slow computing
#Use feature selection  to select the most importatnt features

select = sklearn.feature_selection.SelectKBest(k=11)
selected_features = select.fit(X2, Y2)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X2.columns[i] for i in indices_selected]

X2 = X2[colnames_selected]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X2, Y2,
                                                    stratify=Y2, 
                                                    test_size=0.20)

In [ ]:
############################### Down Sampling of majority class #####################################

from sklearn.utils import resample

df_original = X_train.reset_index(drop=True).merge(y_train.reset_index(drop=True), left_index=True, right_index=True)

# Separate majority and minority classes
df_majority = df_original[df_original['Loan Status']==0]
df_minority = df_original[df_original['Loan Status']==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=10000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

In [ ]:
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE

X2 = df_downsampled.drop('Loan Status',1)
Y2 = df_downsampled['Loan Status']

sm = SMOTE(random_state=12, ratio = 1.0)
X_train, y_train = sm.fit_sample(X2, Y2)

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import svm as SVM
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer

cv = StratifiedKFold(n_splits=10, random_state=42)

In [ ]:
################################### SVC Linear ##############################################
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import numpy
#param values
penal=["l2"]
losses=["hinge","squared_hinge"]
a=numpy.arange(.0000000001, .0001, .00005)
tolerance=a.tolist()
c_regularization=range(10,5000,10)
intercept_scale=range(1,100,4)
max_iters=range(100,4000,100)
#all param-val dictionary
#grid_params_lr = dict('C':[C_regularization], 'penalty':["l1","l2"], 'intercept_scaling':[ntercept_scal_vals], 'max_iter':[max_iter_vals], 'solver' :["newton-cg", "llbfgs", "sag"])
grid_params_svm = dict(penalty=penal, loss=losses, tol=tolerance, C=c_regularization, max_iter=max_iters)
#creating  grid instance  
#KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2)
svm = LinearSVC(fit_intercept=False)
#neigh_grid=GridSearchCV(knn,grid_params_nn,cv=10)
svm_ins=RandomizedSearchCV(svm,grid_params_svm,cv=10,scoring=None,n_iter=1000)
svm_ins.fit(X_train, y_train)

In [ ]:
svm_ins.best_estimator_

In [ ]:
svm_ins.best_score_

In [ ]:
from sklearn.metrics import classification_report

####### i want to use svm_ins.best_estimator_ ################

y_pred = svm_ins.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))